## Ellipsoid Model: Nuclear Deformation

### Import the necessary packages

In [1]:
import parmed as pmd
import json
import sys
from sys import platform
import mdtraj as md
import simtk.openmm.app as mmapp
import mdtraj.reporters
import simtk.unit as u
import random
from openNucleome import OpenNucleome
import warnings
import numpy as np
import pandas as pd

### Important parameters
We defined the important parameters in the next block. First, we set the transition probability between dP particles and P particles as 0.2, and the transition frequency as 4000. When creating the system, we set type 6, 7 as the dP and P particles, respectively, so we kept using 6, 7 here. In this example, we ran a simulation with total length of 100,000 steps, and output one configuration and the energy every 2000 steps.

In [2]:
prob_P_dP = 0.2 # Transition probability from P to dP
prob_dP_P = 0.2 # Transition probability from dP to P
transition_freq = 4000
sampling_freq = 2000
dP_type = 6
P_type = 7
total_steps = 100000

### Initialize the system
We first set up an example "model" of class "OpenNucleome" with the conserved temperature, damping coefficient, timestep, and the mass scale. In this folder, we also included the initial configuration "human.pdb" used for the simulation and created a system according to the initial configuration.

In this example, we moved all the lamina beads, and would consider the dynamics of the membrane, and that is why we set "True" (on) for membrane dynamics. Consequently, we need to designate the bonds between specific lamina beads, so we included a text file, which logs all the pairs linked by the bonds, for the variable "membrane_bond".

In [3]:
model = OpenNucleome(1.0, 0.1, 0.005, 1.0) # 1.0: temperature (LJ reduced unit); 
                                           # 0.1: damping coefficient (LJ reduced unit);
                                           # 0.005: timestep (LJ reduced unit);
                                           # 1.0: mass_scale
    
PDB_file = "human.pdb" #The initial configuration

# Generate new elements and construct topology as well
# flag_membrane: True for including lamina dynamics, False for excluding lamina dynamics;
# lam_bond: A file contains the lamina bond when membrane_dynamics is on.

warnings.filterwarnings("ignore")
model.create_system(PDB_file, flag_membrane = True, lam_bond = 'lamina_bond.txt') 

### Add the force field
Different from the sphere nucleus situation, we used a csv file to log all the flags of interactions between chromosomes and chromosomes, or chromosomes and nuclear landmarks.

The column from "bond" to "lam_squeeze" is boolen, which means the switches of the corresponding interactions. For example, if the spec-chrom is True, it means in the simulation, the interaction between speckle and chromosome will be present; if the inter is False, it means we exclude the interchromosomal interactions between chromosome. Remember some potentials require the corresponding parameter files, such as ideal requires ideal_param_file and spec-chrom requires the chr_spec_param, so if you turn those potentials on, you should also include the corresponding parameter files.

In this example, we turn every interaction on, and set the strength of force squeezing the nucleus as 1.0 (k) when loading the customized force field. The users can set their specific strengths.

Because during the simulations, we would transit the dP speckle and P speckle particles, here, we logged the start index and end index of speckle particle, and computed the number of speckle particles.

In [4]:
# Add the default force field
# Add the customized force field
# In this customized setting, I logged all the flags for different interactions
# (True means on, False means off, and NaN means N/A)
# and all the corresponding files required for interactions. 
force_field = pd.read_csv('input.csv', sep=' ', header=0, index_col=0)
model.load_customized_settings(force_field, k = 1.0)

index_spec_spec_potential = 6
start_spec_index = model.N_chr_nuc+1
end_spec_index = model.N_chr_nuc_spec+1
N_spec = end_spec_index-start_spec_index

force_field

,bond,angle,softcore,ideal,compt,inter,spec-spec,spec-chrom,nuc-nuc,nuc-spec,...,lam-chrom,hard-wall,lam-lam,lam_squeeze,ideal_param_file,compt_param_file,interchr_param_file,chr_spec_param,chr_nuc_param,chr_lam_param
chromosome,True,True,True,True,True,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ideal_param_file.txt,compt_param_file.txt,interchr_param_file.txt,NaN,NaN,NaN
speckle,NaN,NaN,NaN,NaN,NaN,NaN,True,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr_spec_param.txt,NaN,NaN
nucleolus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr_nuc_param.txt,NaN
lamina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,True,True,True,NaN,NaN,NaN,NaN,NaN,chr_lam_param.txt


### Perform the simulation
We first created the simulation with a specific Platform, and here, we used "CUDA" but users can also use "CPU", "Reference", and "OpenCL" according to their hardware. Before the simulation, we minimized the energy to make the system much more reasonable and stable. After randomly setting velocity, we started our simulation with a total length of 100,000 steps and output the configuration and energy every 2000 steps, and change the speckle types every 4000 steps as we mentioned in the previous blocks.

In [5]:
#model.save_system("model_before_simulation_0.xml")

simulation = model.create_simulation(platform_type = "CUDA") # Users can also use CPU, Reference, OpenCL.
simulation.context.setPositions(model.chr_positions)

simulation.minimizeEnergy()

simulation.reporters.append(mdtraj.reporters.DCDReporter('step_100000.dcd', sampling_freq))

def setVelocity(context):
    sigma = u.sqrt(1.0*u.kilojoule_per_mole / model.chr_system.getParticleMass(1))
    velocs = u.Quantity(1.0 * np.random.normal(size=(model.chr_system.getNumParticles(), 3)),
                        u.meter) * (sigma / u.meter)
    context.setVelocities(velocs)
setVelocity(simulation.context)

simulation.reporters.append(mmapp.statedatareporter.StateDataReporter(sys.stdout, sampling_freq, step=True,
    potentialEnergy=True, kineticEnergy=True, temperature=True, progress=True,
    remainingTime=True, separator='\t', totalSteps = total_steps))

for i in range(total_steps//transition_freq):
    simulation.step(transition_freq)
    # Change the type of speckles every 4000 steps, non-equilibrium scheme.

    # Do the chemical modification, and change the spec-spec potential on the fly.
    for j in np.random.randint(start_spec_index-1, end_spec_index-1, N_spec): 

        if model.compart_type[j] == dP_type-1:
            model.compart_type[j] = P_type-1 if random.random() < prob_dP_P else dP_type-1
        else:
            model.compart_type[j] = dP_type-1 if random.random() < prob_P_dP else P_type-1

    # Update the context after changing the type of speckles.
    for m in range(model.chr_system.getNumParticles()):
        model.chr_system.getForce(index_spec_spec_potential).setParticleParameters(m, [model.compart_type[m]])
    model.chr_system.getForce(index_spec_spec_potential).updateParametersInContext(simulation.context)

# Keep the final result of spec types in case constructing the configuration for the continuous simulation.
np.savetxt('compt_final_frame.txt', (np.array(model.compart_type)+1).reshape((-1,1)), fmt='%d')

#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Kinetic Energy (kJ/mole)"	"Temperature (K)"	"Time Remaining"
2.0%	2000	117616.47977582889	99839.74559860234	113.48285494230906	--
4.0%	4000	134255.17869981972	98505.69644786902	111.96650786679427	9:10
6.0%	6000	139626.5962219246	103363.05068186947	117.48761995132067	8:43
8.0%	8000	138422.31900849155	106160.78815787584	120.66766848061837	8:26
10.0%	10000	136876.3147479449	107124.04389536048	121.76255319285312	8:17
12.0%	12000	135761.08125620737	106116.78953242837	120.61765743943052	8:07
14.0%	14000	134950.05574804617	106592.04317908129	121.15785453549233	7:57
16.0%	16000	135071.53483395698	105517.58794190043	119.93657490289775	7:46
18.0%	18000	135336.79790046596	106493.16693579014	121.04546684552605	7:35
20.0%	20000	135073.43377513258	106480.33326483476	121.03087945239244	7:24
22.0%	22000	135995.87157752056	105568.72676501282	119.99470185034126	7:13
24.0%	24000	134968.8553694152	106715.20231363198	121.29784337578074	7:02
26.0%	26000	1